In [44]:
import pandas as pd
import glob
import os

In [45]:
df = pd.read_csv('data/2019-Oct.csv')
#path = r'data/'
#all_files = glob.glob(os.path.join(path, "*.csv"))
#df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df.shape

(42448764, 9)

In [46]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [47]:
df.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [48]:
df.category_code = df.category_code.fillna('sin_dato')
df.brand = df.brand.fillna('sin_dato')

In [49]:
df.event_type.value_counts(normalize=True)

view        0.960673
cart        0.021827
purchase    0.017500
Name: event_type, dtype: float64

In [50]:
df.product_id.nunique()

166794

In [51]:
df.category_id.nunique()

624

In [52]:
df.category_code.nunique()

127

In [53]:
prices = df[['product_id','price']].groupby(['product_id']).nunique()
prices

,price
product_id,
1000978,4
1001588,35
1001606,1
1002042,2
1002062,2
...,...
60500006,1
60500007,2
60500008,1


In [54]:
prices.describe()

,price
count,166794.000000
mean,2.223821
std,4.011440
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,98.000000


In [55]:
prices.quantile(0.90)

price    3.0
Name: 0.9, dtype: float64

In [56]:
prices.quantile(0.97)

price    10.0
Name: 0.97, dtype: float64

In [57]:
category = df[['product_id','category_id']].groupby(['product_id']).nunique()
category

,category_id
product_id,
1000978,1
1001588,1
1001606,1
1002042,1
1002062,1
...,...
60500006,1
60500007,1
60500008,1


In [58]:
category.describe()

,category_id
count,166794.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [59]:
brand = df[['product_id','brand']].groupby(['product_id']).nunique()
brand

,brand
product_id,
1000978,2
1001588,1
1001606,1
1002042,1
1002062,1
...,...
60500006,1
60500007,2
60500008,1


In [60]:
brand.describe()

,brand
count,166794.000000
mean,1.046423
std,0.211592
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


In [61]:
events = df[['event_type','product_id','user_session']].groupby(['product_id','event_type']).count().reset_index()
events

,product_id,event_type,user_session
0,1000978,purchase,12
1,1000978,view,3153
2,1001588,purchase,12
3,1001588,view,2264
4,1001606,view,16
...,...,...,...
220795,60500007,view,12
220796,60500008,view,4
220797,60500009,purchase,1
220798,60500009,view,6


In [62]:
df_views = events[events.event_type == 'view']
df_views = df_views.rename(columns={'user_session': 'cant_views'})
print('cantidad productos con views: ', df_views.product_id.nunique())
df_views.head()

cantidad productos con views:  166794


,product_id,event_type,cant_views
1,1000978,view,3153
3,1001588,view,2264
4,1001606,view,16
7,1002042,view,320
9,1002062,view,1556


In [63]:
df_clicks = events[events.event_type == 'cart']
df_clicks = df_clicks.rename(columns={'user_session': 'cant_clicks'})
print('cantidad productos con clicks: ', df_clicks.product_id.nunique())
df_clicks.head()

cantidad productos con clicks:  11765


,product_id,event_type,cant_clicks
5,1002042,cart,5
45,1002524,cart,10300
52,1002528,cart,609
57,1002532,cart,1673
62,1002535,cart,43


In [64]:
df_purchase = events[events.event_type == 'purchase']
df_purchase = df_purchase.rename(columns={'user_session': 'cant_purchase'})
print('cantidad productos con clicks: ', df_purchase.product_id.nunique())
df_purchase.head()

cantidad productos con clicks:  42241


,product_id,event_type,cant_purchase
0,1000978,purchase,12
2,1001588,purchase,12
6,1002042,purchase,3
8,1002062,purchase,18
10,1002098,purchase,13


In [65]:
df_events = df_views.merge(df_clicks[['product_id','cant_clicks']],on='product_id',how='left').merge(df_purchase[['product_id','cant_purchase']],on='product_id',how='left')
df_events = df_events.drop(columns=['event_type'])
df_events[['cant_clicks','cant_purchase']] = df_events[['cant_clicks','cant_purchase']].fillna(0)
df_events['ctr'] = df_events.cant_clicks / df_events.cant_views
df_events['cvr'] = df_events.cant_purchase / df_events.cant_views
df_events

,product_id,cant_views,cant_clicks,cant_purchase,ctr,cvr
0,1000978,3153,0.0,12.0,0.000000,0.003806
1,1001588,2264,0.0,12.0,0.000000,0.005300
2,1001606,16,0.0,0.0,0.000000,0.000000
3,1002042,320,5.0,3.0,0.015625,0.009375
4,1002062,1556,0.0,18.0,0.000000,0.011568
...,...,...,...,...,...,...
166789,60500006,14,0.0,0.0,0.000000,0.000000
166790,60500007,12,0.0,0.0,0.000000,0.000000
166791,60500008,4,0.0,0.0,0.000000,0.000000
166792,60500009,6,0.0,1.0,0.000000,0.166667


In [66]:
df_events.describe()

,product_id,cant_views,cant_clicks,cant_purchase,ctr,cvr
count,1.667940e+05,166794.000000,166794.000000,166794.000000,166794.000000,166794.000000
mean,1.970715e+07,244.489604,5.554840,4.453691,0.001786,0.005676
std,1.187490e+07,2717.840887,226.338216,129.900205,0.009802,0.022366
min,1.000978e+06,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.090023e+07,7.000000,0.000000,0.000000,0.000000,0.000000
50%,1.830011e+07,25.000000,0.000000,0.000000,0.000000,0.000000
75%,2.640381e+07,91.000000,0.000000,1.000000,0.000000,0.001779
max,6.050001e+07,419287.000000,52123.000000,28944.000000,0.500000,1.000000


In [67]:
df_events.dtypes

product_id         int64
cant_views         int64
cant_clicks      float64
cant_purchase    float64
ctr              float64
cvr              float64
dtype: object

In [68]:
print('% productos con ctr mayor a 0: ', 100*((df_events[df_events.ctr > 0].product_id.nunique())/ df_events.product_id.nunique()))
print('% productos con cvr mayor a 0: ', 100*((df_events[df_events.cvr > 0].product_id.nunique())/ df_events.product_id.nunique()))

% productos con ctr mayor a 0:  7.053611041164551
% productos con cvr mayor a 0:  25.325251507848005


In [69]:
data = df.merge(df_events,on='product_id',how='left')
data['gmv'] = data.price * data.ctr * data.cvr
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,cant_views,cant_clicks,cant_purchase,ctr,cvr,gmv
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,sin_dato,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,143,0.0,1.0,0.000000,0.006993,0.000000
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,3019,23.0,27.0,0.007618,0.008943,0.002262
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,sin_dato,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,1181,0.0,5.0,0.000000,0.004234,0.000000
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,17571,6.0,651.0,0.000341,0.037050,0.003185
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,52856,3090.0,2249.0,0.058461,0.042550,2.691402


In [70]:
data.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     2
cant_views       0
cant_clicks      0
cant_purchase    0
ctr              0
cvr              0
gmv              0
dtype: int64

In [72]:
data_final = data[['product_id','category_id','brand','price','cant_views','cant_clicks','cant_purchase','ctr','cvr','gmv']].drop_duplicates()
data_final.head()

,product_id,category_id,brand,price,cant_views,cant_clicks,cant_purchase,ctr,cvr,gmv
0,44600062,2103807459595387724,shiseido,35.79,143,0.0,1.0,0.000000,0.006993,0.000000
1,3900821,2053013552326770905,aqua,33.20,3019,23.0,27.0,0.007618,0.008943,0.002262
2,17200506,2053013559792632471,sin_dato,543.10,1181,0.0,5.0,0.000000,0.004234,0.000000
3,1307067,2053013558920217191,lenovo,251.74,17571,6.0,651.0,0.000341,0.037050,0.003185
4,1004237,2053013555631882655,apple,1081.98,52856,3090.0,2249.0,0.058461,0.042550,2.691402


In [73]:
data_final.shape

(371536, 10)

In [74]:
data_final.product_id.nunique()

166794

In [75]:
data.dtypes

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
cant_views         int64
cant_clicks      float64
cant_purchase    float64
ctr              float64
cvr              float64
gmv              float64
dtype: object

In [76]:
data_final.to_csv('data/data_products.csv', index=False)